In [139]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas
import time
import re
import os
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [141]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
# options.add_argument('Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/13.0 MQQBrowser/10.1.1 Mobile/15B87 Safari/604.1 QBWebViewUA/2 QBWebViewType/1 WKType/1')
chrome = webdriver.Chrome('./chromedriver', options=options)
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    print(url.format(i,i))
    time.sleep(5)
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card])   
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=1&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_1
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=2&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_2
總共耗時:22.174534 秒


In [142]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
print(amazon_json)

{'sheet_name': '顯示卡', 'title': ['MSI', 'XFX', 'PowerColor', 'NVIDIA', 'ASUS', 'ZOTAC', 'GIGABYTE', 'Gigabyte', 'Sapphire', 'MSI微星', 'EVGA']}


In [212]:
title=[]
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    # 分種類並寫入json中
    for aaa in columns_of_datas:
    # 正則表達式將標題的牌子切割出來
        matchObj = re.split(r'\s', df['f'][aaa], re.M|re.I)
        title.append(matchObj[0])
# 篩選重複的牌子        
json_title_dumps = list(set(title))
json_title = {'title': json_title_dumps}
amazon_json.update(json_title)
# 將整理好title的json寫入json檔案中
with open('AMAZON_COMPUTER_PRICE.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_json, f, ensure_ascii=False, indent=4,sort_keys=True)
print(amazon_json)

{'sheet_name': '顯示卡', 'title': ['MSI', 'XFX', 'PowerColor', 'NVIDIA', 'ZOTAC', 'GIGABYTE', 'Gigabyte', 'Sapphire', 'MSI微星', 'EVGA']}


In [217]:
# 看這裡
for adx,datas in enumerate(data):
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    #   每頁資料的數量
    columns_of_datas = [ddd for ddd in range(0,len(datas))]    
    #   將每頁資料歸納種類
    for aaa in columns_of_datas:        
        if any(x in df['f'][aaa] for x in amazon_json['title']):
            print(df['f'][aaa])
        

XFX HDMI 顯卡
XFX HDMI 顯卡
MSI微星 Gaming GeForce RTX 2060 6GB GDRR6 192 位元 HDMI/DP 光線追蹤圖靈架構VR Ready 顯卡(RTX 2060 VENTUS XS 6G OC)
EVGA 06G-P4-1068-KR GeForce GTX 1660 Super Sc Ultra Gaming, 6GB GDDR6, Dual Fan, Metal Backplate
XFX RX 5700 Xt Thicc III Ultra 8GB Boost 高達 2025MHz GDDR6 3xDP HDMI 顯示卡(Rx-57XT8TBD8)
MSI Gaming GeForce GTX 1660 192-Bit HDMI/DP 6GB GDRR5 HDCP Support DirectX 12 Dual Fan VR Ready OC Graphics Card (GTX 1660 VENTUS XS 6G OC) Socket AM2 插槽 ATX 主機板
Sapphire 11265-05-20G Radeon Pulse RX 580，8GB，GDDR5（第五版圖形用雙倍資料傳輸率記憶體），雙 HDMI / DVI-D /雙 DP，超頻版本，帶背板（UEFI），PCI-E 接口 顯示卡
Sapphire 11293-01-20G Radeon PULSE RX 5700 Xt 8GB GDDR6 HDMI/ TRIPLE DP OC 附背板(UEFI)PCIe 4.0 顯示卡
EVGA 06G-P4-2068-KR GeForce RTX 2060 KO Ultra Gaming,6GB GDDR6,雙風扇,金屬背板
XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-57XT83LD8
MSI微星 Gaming GeForce GT 710 2GB GDRR3 64 位元 HDCP 支援 DirectX 12 OpenGL 4.5 單風扇低調顯卡(GT 710 2GD3 LP)
MSI微星 Gaming GeForce RTX 2080 Super 8GB GDRR6 256-Bit HDMI/DP Nvlink Tri-Frozr 圖靈架構超頻顯示卡

In [9]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "D:\\Amazon_graphics_card\\"
if os.path.exists(output_file) == False:
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book.create_sheet('default')  # 新增空sheet
else:  # 如果檔案存在，append
    book = load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book = book
    
if amazon_json['sheet_name'] in writer.book.sheetnames:
    writer.book.remove(writer.book[amazon_json['sheet_name']])
    

for adx,datas in enumerate(data):
    df = "df" + str(adx)    
    df = pd.DataFrame(datas) 
    df.to_excel(writer, index=False) 


# 寫入excel sheet
  
if (len(writer.book.sheetnames) > 1):  # 若有其他sheet，刪除default sheet
    if 'default' in writer.book.sheetnames:
        writer.book.remove(writer.book['default'])    
writer.save()  


                                                    0
0                                         XFX HDMI 顯卡
1   XFX RX 5700 Xt Thicc III Ultra 8GB Boost 高達 20...
2   MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...
3                                         XFX HDMI 顯卡
4   MSI Gaming GeForce GTX 1660 192-Bit HDMI/DP 6G...
5   Sapphire 11265-05-20G Radeon Pulse RX 580，8GB，...
6   Sapphire 11293-01-20G Radeon PULSE RX 5700 Xt ...
7   XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-...
8   Gigabyte 技嘉 Gv-R57XTGAMING OC-8GD Radeon Rx 57...
9   GIGABYTE GeForce GTX 1660 Ti OC 6G 192-bit GDD...
10  MSI Gaming GeForce RTX 2060 Super 8GB GDRR6 25...
11  PowerColor Red Devil Radeon RX 5700 XT 8GB GDD...
12  MSI Gaming GeForce RTX 2080 Super 8GB GDRR6 25...
13  Gigabyte 技嘉 Gv-N1660OC-6GD GeForce GTX 1660 OC...
14  MSI微星 Gaming GeForce GT 710 2GB GDRR3 64 位元 HD...
15  EVGA 06G-P4-1068-KR GeForce GTX 1660 Super Sc ...
                                                    0
0   EVGA GeForce RTX 2080 Su

In [5]:
cwd = os.getcwd()
curr_file = cwd + "\\" + output_file
target_file = target_path + output_file

try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200909_amazon_graphics card.xlsx
移動檔案完畢.
